## Table of Contents

1. [Introduction](#introduction)
2. [Preparing the Data](#preparing-the-data)  
   2.1 [Getting the Data](#getting-the-data)  
   2.2 [Cleaning the Data](#cleaning-the-data)  
   2.3 [Preprocessing](#preprocessing)  
       $\;\;\;$ 2.3.1 [Normalisation of Numerical Features](#normalisation-of-numerical-features)  
3. [Simple k-Nearest Neighbours](#simple-k-nearest-neighbours)  
   3.1 [Implementing simple kNN](#implementing-simple-knn)  
   3.2 [Tuning k and lambda via T-fold Cross Validation](#tuning-k-and-lambda)  
   3.3 [Testing and Analysing the Model with Optimal Hyperparameters](#testing-and-analysing-the-model)  
       $\;\;\;$ 3.3.1 [Discussion](#discussion)
4. [Weighted kNN](#weighted-knn)<br>
   4.1 [Implementing Weighted kNN](#implementing-weighted)<br>
   4.2 [Tuning k, lambda and the Weights via T-fold Cross Validation](#tuning-k-and-lambda-and-the-weights)<br>
   4.3 [Discussion](#discussion-two)
5. [Two-step kNN](#two-step-knn)<br>
   5.1 [Implementing Two-step kNN](#implementing-two-step)<br>
   5.2 [Tuning k1,k2,lambda1, lambda2 and the Weights](#tuning)<br>
   5.3 [Discussion](#discussion-three)

# Introduction

The dataset contains data about tumors of different patients. The feature space consists of characteristics such as the total surface, density, diameter, etc. Each tumor belongs to one of three classes: $0,1,2$. Class $0$ is benign tumors, Class $1$ is a malignant type glioma, and Class $2$ is a malignant type meningioma. 

We will train a simple kNN, a weighted kNN and a two-step kNN (from scratch) on this data to predict the classes of future tumors. In order to assess their accuracy, we will use typical classification tools such as precision, F-score, accuracy, etc. We also use T-fold cross validation to tune the hyperparameter $k$.

We outline two goals:

1. Distinguish Class $0$ from Classes $1,2$ (benign vs malignant).
2. Accuractely predict Class $2$, the minority class in this imbalanced dataset.

# Preparing the Data

## Getting the Data

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [7]:
# Load the data
train = pd.read_csv('brain_cancer_samples.csv')
test = pd.read_csv('brain_cancer_test.csv')

In [8]:
# Inspect
train.head(10)

,Patient ID,2D Surface,Contour Size,Total Surface,Density,Diameter,Primary Axis,Shape Variation,Location,Eccentricity_Real,Eccentricity_Imag,Class
0,1,1325.5,160.124890,1518.5,0.872901,41.081371,36.290924,53.992371,2,6.750000e-17,1.101565,1
1,2,81.5,39.556349,88.0,0.926136,10.186708,8.801125,15.591203,2,8.950000e-17,1.462264,1
2,3,57177.0,912.607209,57611.0,0.992467,269.814784,234.644165,312.589264,4,5.390000e-17,0.880179,0
3,4,135.0,92.083260,206.0,0.655340,13.110581,6.568726,40.183521,3,3.700000e-16,6.035112,0
4,5,6653.0,1525.307779,84025.5,0.079178,92.037290,383.980133,402.407623,3,1.920000e-17,0.313504,1
5,6,101.0,62.284271,173.0,0.583815,11.340070,14.772180,24.742086,0,8.230000e-17,1.343625,0
6,7,11.5,13.071068,12.0,0.958333,3.826520,3.650945,4.760943,4,5.120000e-17,0.836956,1
7,8,104.5,42.727922,109.0,0.958716,11.534883,9.347109,16.334406,1,8.780000e-17,1.433137,2
8,9,1306.0,208.024384,1944.5,0.671638,40.778068,43.150715,54.268845,1,4.670000e-17,0.762695,1
9,10,2062.5,1125.050858,37117.0,0.055568,51.245064,543.773193,940.586609,2,8.640000e-17,1.411383,0


## Cleaning the Data

In [10]:
# Dealing with missing data

l_train = train.shape[0]
l_test = test.shape[0]

train.dropna(inplace=True)
test.dropna(inplace=True)

print(f"There were {l_train - train.shape[0]} patients with missing data in the training set and {l_test - test.shape[0]} patients with missing data in the test set.")

There were 0 patients with missing data in the training set and 0 patients with missing data in the test set.


In [11]:
# Dealing with duplicate entries

train = train.drop_duplicates(subset=["Patient ID"], keep="first")
test = test.drop_duplicates(subset=["Patient ID"], keep="first")

print(f"There were {l_train - train.shape[0]} duplicate patients in the training set and {l_test - test.shape[0]} duplicate patients in the test set." )

There were 0 duplicate patients in the training set and 0 duplicate patients in the test set.


In [12]:
# Patient ID column is unnecessary

train.drop(columns = ["Patient ID"], inplace = True)
test.drop(columns = ["Patient ID"], inplace = True)

In [13]:
# Check if there are incorrect records of the classes

print(train["Class"].isin([0,1,2]).all(), test["Class"].isin([0,1,2]).all())

True True


## Preprocessing

In [15]:
# Divide the data into features and outputs, as well as collect the feature names
column_names = list(train)
target = "Class"

X_train, y_train, X_test, y_test = train.drop(target, axis=1), train[target], test.drop(target, axis=1), test[target]

We have all numerical features in completely different scales. We have to normalise these features, or else the kNN model would emphasise those with larger scale. 

Additionally, the location is a categorical variable, so we do not include it in the normalisation. It has the following unique values:

In [17]:
print(pd.unique(train["Location"]), pd.unique(test["Location"]))

[2 4 3 0 1] [1 0 4 3 2]


### Normalisation of Numerical Features

In [19]:
# Categorical feature
location_train = X_train["Location"]
location_test = X_test["Location"]

# Numerical features
X_train_num = X_train.drop(columns = ["Location"])
X_test_num = X_test.drop(columns = ["Location"])

# Normalisation
mean = X_train_num.mean()
std = X_train_num.std()

X_train_norm = (X_train_num - mean)/std
X_test_norm = (X_test_num - mean)/std

In [20]:
# Putting it all back together with Location at the end

X_train = pd.concat([X_train_norm, location_train], axis=1)
X_test = pd.concat([X_test_norm, location_test], axis=1)

In [21]:
X_test.head()

,2D Surface,Contour Size,Total Surface,Density,Diameter,Primary Axis,Shape Variation,Eccentricity_Real,Eccentricity_Imag,Location
0,-0.298128,-0.779382,-0.403209,1.152581,-0.742030,-0.689443,-0.499074,-0.423931,-0.423531,1
1,0.332066,1.267877,0.222724,-0.019909,1.058213,0.217939,0.083884,-0.310857,-0.310389,0
2,-0.298275,-0.775000,-0.403282,0.843197,-0.751160,-0.703162,-0.490741,0.794162,0.794127,4
3,-0.204848,0.971321,0.473125,-1.803678,-0.077614,0.886545,0.299893,-0.493930,-0.493259,3
4,-0.241829,-0.372979,-0.356415,0.373062,-0.235757,-0.456187,-0.334891,-0.257012,-0.256545,3


# Simple k-Nearest Neighbours

## Implementing simple kNN

We implement the $k$-nearest neighbours method on 3 classes. This model makes a prediction on an unseen data point as follows: find the $k$ nearest neighbours to the point in the training set, then count how many intances of each class are in this neighborhood, then assign the class with most instances to the point. The hyperparameter $k$ will be tuned with cross validation. We will then test the model according to various metrics.

In order to apply the kNN, we need a metric to determine the k-nearest neighbours. We use Euclidean distance for the numerical features and adapt it to include the categorical variable (Location). This adaptation introduces a new hyperparameter $\lambda$, which will also be tuned with T-fold cross validation.

In [25]:
def adapted_euclidian_distance(p, q, lamb):
    numerical = np.sqrt(np.sum((p[0:-1]-q[:, 0:-1])**2, axis=1)) # Euclidean distance for numerical variables
    categorical = lamb*(p[-1] != q[:,-1])                        # Less distance if same location, more distance if different location 
                                                                 # (weighted by lambda)
    return numerical + categorical

Given training and test sets, we can thus find the k-nearest neighbours in the training set for every point in the test set.

In [27]:
def k_neighbours(X_train, X_test, k, lamb, return_distance=False):
    n_neighbours = k
    neigh_ind = [] # list of lists (ith entry is the indices of the k closest neighbours in the training set of the ith x_test)
    dist = []      # list of lists (stores distances of the neighbours above)
    

    # Compute distance from every point x_test in X_test to every point in X_train 
    # (ith entry is a list of length len(X_train.shape[0]) with the distances of all training points to the ith x_test)
    point_dist = [adapted_euclidian_distance(x_test, X_train, lamb) for x_test in X_test] 

    # Determine which k training points are closest to each test point
    for row in point_dist: 
        enum_neigh = enumerate(row)
        sorted_neigh = sorted(enum_neigh, key=lambda x: x[1])[:k] # Sort to get k closest

        ind_list = [tup[0] for tup in sorted_neigh]  # Indices of the k closest
        dist_list = [tup[1] for tup in sorted_neigh] # Distances to the point at indices above

        dist.append(dist_list)     # Store distances
        neigh_ind.append(ind_list) # Store indices of neighbours

    
    # Option to return distances too
    if return_distance:
        return np.array(dist), np.array(neigh_ind)

    return np.array(neigh_ind)

We can now predict the labels from a test dataset by getting the k-nearest neighbours for each point, and then assigning the class by majority vote.

In [29]:
def predict_kNN(X_train, y_train, X_test, k, lamb):
    neighbours = k_neighbours(X_train, X_test, k, lamb)
    
    # Count number of occurences of label with np.bincount and choose the label that has most with np.argmax 
    y_pred = np.array([np.argmax(np.bincount(y_train[row])) for row in neighbours]) 
    
    # Note: when given a pandas Series, np.bincount ignores the index column and deals with the values column
    # almost as if it had been given as an array: for each distinct value, it says how many times it appears 
    # by returning an array in ascending order (so if there are 2 0s, 3 1s and 1 2, it returns 2,3,1)

    return y_pred

We try this model with an arbitrary choice of $k$ and $\lambda$.

In [31]:
k=6
lamb = 0.5
prediction = predict_kNN(X_train.to_numpy(), y_train.to_numpy(), X_test.to_numpy(), k, lamb)
prediction

array([1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 0, 1,
       1, 0, 0, 0, 0, 2, 1, 1, 2, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1,
       0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1], dtype=int64)

## Tuning $k$ and $\lambda$ via T-fold Cross Validation
<a id="tuning-k-and-lambda"></a>

T-fold cross validation for a single hyperparameter works as follows. We first fix a value for the hyperparameter. Then we split our training data randomly into T subsets ("folds"). We test how good the model is at predicting out of sample by training it on the first $T-1$ subsets and then testing it on the remaining subset (with a suitable measure of error). Then we repeat this process for each of the $T$ subsets, and finally average the $T$ error measures. This final error measure is a robust indicator of how good the model is at predicting out of sample with this specific value of the hyperparameter.

By doing this with multiple values of the hyperparameter, we can choose the value which gives our model the best generalisability. In our case, we have two hyperparameters. So we will be checking for pairs of values $(k, \lambda)$.

Note: $T$ is typically chosen as $5$ or $10$, depending on the size of the dataset. In our case, $T=5$.

The first task is to create the measure of error.

In [34]:
print(f"There are {len(train[train["Class"] == 0])} samples of Class 0, {len(train[train["Class"] == 1])} of Class 1, and {len(train[train["Class"] == 2])} of Class 2.")

There are 47 samples of Class 0, 80 of Class 1, and 18 of Class 2.


So we have an imbalanced dataset. It is very important that we correctly predict all the cases for Classes $1,2$, since these are malignant tumors. We therefore avoid measures such as accuracy or specificity. We also prefer the macro-averaged versions of the measures over the micro-averaged versions, since the former give equal weight to all classes regardless of their size. We will use macro-averaged recall:

$$
\frac{1}{3}\sum_{i \in \{0,1,2\}}{\textnormal{Recall}_{i}} \textnormal{ ,}
$$

where

$$
\textnormal{Recall}_{i} = \frac{TP_i}{TP_i + FN_i}.
$$

In [36]:
# Computing the confusion matrix

def confusion_matrix(y_true, y_pred):
    # Given the true and predicted values, gives the confusion matrix with truth 
    # as the rows and predictions as columns
    return pd.crosstab(y_true, y_pred, rownames=['Truth'], colnames=['Predicted'])

In [37]:
print(f"The confusion matrix is \n \n {confusion_matrix(y_test, prediction)}")

The confusion matrix is 
 
 Predicted   0   1  2
Truth               
0          12   6  0
1           7  28  0
2           0   4  3


In [38]:
# Computing macro-averaged recall

def macro_recall(y_true, y_pred):
    # Get confusion matrix and initialise macro_recall
    conf_matrix = confusion_matrix(y_true, y_pred)
    macro_recall = 0

    # It is possible that there were no predictions made of a class
    # To avoid the matrix not being 3x3, we define the missing columns here 
    if {0,1,2} != set(conf_matrix.columns):
        # Missing classes
        missing = {0,1,2} - set(conf_matrix.columns)
        for cls in missing:
            # Add a column for missing class with no predictions
            conf_matrix[cls]=[0,0,0]
            

    # Compute recall for each class and sum
    for i in range(len(conf_matrix)):
        TP = conf_matrix.iloc[i,i]
        FN = conf_matrix.iloc[i,:].sum() - TP
        macro_recall += TP/(TP+FN)
        
    return macro_recall/len(conf_matrix)

In [39]:
print(f"The macro-averaged recall is {macro_recall(y_test, prediction)}.")

The macro-averaged recall is 0.6317460317460318.


Now we can perform 5-fold cross validation. We first split the data:

In [41]:
# Splitting the data into T=5 subsets or "folds" (getting the indices of each fold)

T=5
folds_indexes = np.split(np.arange(len(y_train)), T)
folds_indexes

[array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28]),
 array([29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45,
        46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57]),
 array([58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74,
        75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86]),
 array([ 87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,  98,  99,
        100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112,
        113, 114, 115]),
 array([116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128,
        129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141,
        142, 143, 144])]

In [42]:
def cross_validation_score_kNN(X_train, y_train, folds_indexes, k, lamb):
    # Compute the macro-averaged accuracy for a specific pair (k,lamb) for every fold
    # and then average these scores. Gives a sense of how well this pair of hyperparameters
    # predicts out-of-sample.

    # Store the scores for each fold
    scores = [] 

    # Iterate over the folds
    for i in range(len(folds_indexes)): 
        # The indices for that fold (what we will validate on)
        val_indexes = folds_indexes[i] 

        # Get the indices of all the other folds (what we will train on)
        train_indexes = list(set(range(y_train.shape[0])) - set(val_indexes)) 

        # Get the training data
        X_train_i = X_train[train_indexes, :] 
        y_train_i = y_train[train_indexes]

        # Get the validation data
        X_val_i = X_train[val_indexes, :] 
        y_val_i = y_train[val_indexes] 

        # Make a prediction on the validation fold
        prediction = predict_kNN(X_train_i, y_train_i, X_val_i, k, lamb)

        # Store the macro-averaged recall score for this prediction
        score_i = macro_recall(y_val_i, prediction)
        scores.append(score_i)

    # Return the average score
    return sum(scores) / len(scores) 

In [43]:
cross_validation_score_kNN(X_train.to_numpy(), y_train.to_numpy(), folds_indexes, k, lamb)

0.7278604339130654

We now scan ranges of $k$ and $\lambda$ to find the optimal pair of hyperparameters:

In [45]:
def choose_best_k_and_lamb(X_train, y_train, folds_indexes, k_range, lamb_range):
    best_score = 0

    for i, k in enumerate(k_range):
        for j, lamb in enumerate(lamb_range):
            score = cross_validation_score_kNN(X_train, y_train, folds_indexes, k, lamb)
            if score > best_score:
                best_score = score
                best_params = [k,lamb]
            print(f'The cross validation score for the pair {k,lamb}: {score:.3f}')
    
    return best_score, best_params

In [46]:
k_range=np.arange(1, 31) #1,2,...,30
lamb_range=np.arange(0,5.1,0.1) #0,0.1,0.2,...,4.9,5

best_values = choose_best_k_and_lamb(X_train.to_numpy(), y_train.to_numpy(), folds_indexes, k_range, lamb_range)
print(f'Best set of values is {best_values[1]} with score {best_values[0]}.')

The cross validation score for the pair (1, 0.0): 0.615
The cross validation score for the pair (1, 0.1): 0.631
The cross validation score for the pair (1, 0.2): 0.677
The cross validation score for the pair (1, 0.30000000000000004): 0.649
The cross validation score for the pair (1, 0.4): 0.648
The cross validation score for the pair (1, 0.5): 0.652
The cross validation score for the pair (1, 0.6000000000000001): 0.629
The cross validation score for the pair (1, 0.7000000000000001): 0.624
The cross validation score for the pair (1, 0.8): 0.629
The cross validation score for the pair (1, 0.9): 0.637
The cross validation score for the pair (1, 1.0): 0.632
The cross validation score for the pair (1, 1.1): 0.632
The cross validation score for the pair (1, 1.2000000000000002): 0.632
The cross validation score for the pair (1, 1.3): 0.632
The cross validation score for the pair (1, 1.4000000000000001): 0.637
The cross validation score for the pair (1, 1.5): 0.644
The cross validation score f

## Testing and Analysing the Model with Optimal Hyperparameters

With the optimal hyperparameters, we can make the optimal prediction on the test set. This is the resulting confusion matrix:

In [49]:
optimal_k, optimal_lamb = best_values[1][0], best_values[1][1]

optimal_prediction = predict_kNN(X_train.to_numpy(), y_train.to_numpy(), X_test.to_numpy(), optimal_k, optimal_lamb)
confusion_matrix(y_test, optimal_prediction)

Predicted,0,1,2
Truth,,,
0,12,6,0
1,6,29,0
2,0,3,4


Due to the abundance of Class $1$ samples and the lack of Class $2$ samples, Class $2$ is not well predicted: It is often mistaken for Class $1$ (there are many neighbours around which are of Class $1$). The tuning process tried to choose the best $(k,\lambda)$ so that Class $2$ would be well predicted, but choosing the pair that predicts Class $2$ best was probably too extreme for a good prediction of Classes $1,2$. So the total sum of the recalls was worse than just leaving Class $2$ to be badly predicted and predicting Classes $0,1$ reasonably well. This explains the confusion matrix above.

We will now compute other measures of error fitting for this imbalanced dataset to further analyse the prediction:


$$
\textnormal{Accuracy } = \frac{\textnormal{Correct predictions}}{\textnormal{All predictions}} = \frac{\sum_{i \in {0,1,2}} {TP_i + TN_i}}{\sum_{i \in {0,1,2}} {TP_i + TN_i + FN_i + FP_i}}
$$

$$
\textnormal{Macro-averaged precision } = \frac{1}{3} \sum_{i \in {0,1,2}} {\textnormal{Precision}_i} \textnormal{, where } \textnormal{Precision}_i = \frac{TP_i}{TP_i + FP_i}
$$

$$
\textnormal{Macro-averaged F-score } = \frac{1}{3} \sum_{i \in {0,1,2}} {\textnormal{F-score}_i} \textnormal{, where } \textnormal{F-score}_i = 2 \cdot \frac{\textnormal{Precision}_i \cdot \textnormal{Recall}_i}{\textnormal{Precision}_i + \textnormal{Recall}_i}
$$

$$
\textnormal{Macro-averaged recall } = \frac{1}{3}\sum_{i \in \{0,1,2\}}{\textnormal{Recall}_{i}} \textnormal{, where } \textnormal{Recall}_{i} = \frac{TP_i}{TP_i + FN_i}
$$

**Note:** For classification tasks where each sample is assigned one (and only one) class (such as this one), micro-averaged precision, micro-averaged F-score and micro-averaged recall are all equal to accuracy. This is because the total amount of FPs and the total amount of FNs are equal: something that is FP for one class is FN for another. Similarly for the TPs and TNs. So the measures above are a complete list (except for specificity, but we do not care about it in the case of an imbalanced dataset because it depends on the TN).

In [51]:
# Accuracy: how accurate is the model overall?

def acc(y_true, y_pred):
    numerator = np.sum(y_pred == y_true)
    denominator = len(y_pred)
    return numerator/denominator

In [52]:
# Macro-averaged precision: for each class equally weighted, 
# how much can we trust the model when it predicts that a sample corresponds to a class?

def macro_precision(y_true, y_pred):
    # Get confusion matrix and initialise macro_precision
    conf_matrix = confusion_matrix(y_true, y_pred)
    macro_precision = 0

    # It is possible that there were no predictions made of a class
    # To avoid the matrix not being 3x3, we define the missing columns here 
    if {0,1,2} != set(conf_matrix.columns):
        # Missing classes
        missing = {0,1,2} - set(conf_matrix.columns)
        for cls in missing:
            # Add a column for missing class with no predictions
            conf_matrix[cls]=[0,0,0]
            

    # Compute precision for each class and sum
    for i in range(len(conf_matrix)):
        TP = conf_matrix.iloc[i,i]
        FP = conf_matrix.iloc[:,i].sum() - TP
        macro_precision += TP/(TP+FP)
        
    return macro_precision/len(conf_matrix)

In [53]:
# Macro-averaged F-score: for each class equally weighted, are precision and recall similar?

def macro_F(y_true, y_pred):
    # Get confusion matrix and initialise macro_F
    conf_matrix = confusion_matrix(y_true, y_pred)
    macro_F = 0

    # It is possible that there were no predictions made of a class
    # To avoid the matrix not being 3x3, we define the missing columns here 
    if {0,1,2} != set(conf_matrix.columns):
        # Missing classes
        missing = {0,1,2} - set(conf_matrix.columns)
        for cls in missing:
            # Add a column for missing class with no predictions
            conf_matrix[cls]=[0,0,0]
            

    # Compute F-score for each class and sum
    for i in range(len(conf_matrix)):
        TP = conf_matrix.iloc[i,i]
        FP = conf_matrix.iloc[:,i].sum() - TP
        FN = conf_matrix.iloc[i,:].sum() - TP
        precision_i = TP/(TP+FP)
        recall_i = TP/(TP+FN)
        macro_F += 2*precision_i*recall_i/(precision_i + recall_i)
        
    return macro_F/len(conf_matrix)

In [54]:
# Macro-averaged recall: for each class equally weighted,
# how many instances of the class did it correctly detect?

# Implemented with the macro_recall function we defined before

### Discussion

In [56]:
print(f"The accuracy of our model is {acc(y_test, optimal_prediction)}.")

The accuracy of our model is 0.75.


In [57]:
print(f"The macro-averaged recall of our model is {macro_recall(y_test, optimal_prediction)}.")

The macro-averaged recall of our model is 0.6888888888888888.


In [58]:
print(f"The macro-averaged precision of our model is {macro_precision(y_test, optimal_prediction)}.")

The macro-averaged precision of our model is 0.8099415204678362.


In [59]:
print(f"The macro-averaged F-score of our model is {macro_F(y_test, optimal_prediction)}.")

The macro-averaged F-score of our model is 0.7294866472948666.


In [60]:
confusion_matrix(y_test, optimal_prediction)

Predicted,0,1,2
Truth,,,
0,12,6,0
1,6,29,0
2,0,3,4


Recall our two **goals**:

1. Distinguish Class $0$ from Classes $1,2$ (benign vs malignant).
2. Accuractely predict Class $2$, the minority class in this imbalanced dataset.

We will use the measures computed above to assess how well the model satisfies these goals and how we can improve it.
<br>
<br>
<br>

**Accuracy:** This is the most basic metric, and serves only as an overall measure that can be incredibly misleading. In our case, the $75\%$ accuracy does not represent the struggle with correctly identifying Class $2$ and distinguishing benign (Class $0$) from malignant (Classes $1,2$).


**Precision vs recall:** These two metrics push against each other somewhat. Loosely speaking, higher recall means the model is "extra safe" about the minority class (Class $2$), in that it predicts Class $2$ a lot in order to be safe and not miss any Class $2$ samples. In contrast, optimising for precision means leaving behind the safety measures to ensure that we can trust the model more when it predicts a class, to the detriment of the minority class.

**F-score:** The macro-averaged F-score is in between the recall and the precision because it is the middle point in some sense. It rewards models which have similar precision and recall, at the middle point of being safe and not going overboard with safety. In our case, we would not mind a lower F-score if it means a higher recall and lower precision.
<br>
<br>
<br>

The second goal was to predict Class $2$ correctly, so higher recall is what we want. That is why we used it as a measure for the cross validation. However, it seems that even while prioritising recall, Class $2$ is still not well predicted (our recall value is much lower than our precision value). This means that this model is not intrinsically equipped to detect Class $2$ (it lacks the mathematical power and flexibility to detect the subtleties that make Classes $1$ and $2$ differ).

As for the first goal, there has been more success. This has driven the accuracy value up higher than recall (due to the high number of samples of Classes $0$ and $1$). But there is some confusion between Classes $0$ and $1$, and it is very important that the model learns to address this.
<br>
<br>
<br>

**Improvement:**  We will try a **weighted version of the model** above. Even though we consider the $k$ nearest neighbours, it is possible that the ones furthest away are not very relevant. Therefore weighing the importance of each sample by the inverse of the distance could lead to better classification. Additionally, we can add an extra layer of complexity by weighing each class. We tune these weights via $5$-fold cross validation.

# Weighted kNN

## Implementing Weighted kNN

In this section we implement the improvement as explained above.

First we adapt the way we make predictions to weigh each sample:

In [65]:
class_weights = {0: 0.1, 1: 0.2, 2: 0.1} # arbitrary class weights

def predict_kNN_weight(X_train, y_train, X_test, k, lamb, class_weights):
    # Get distances and indices of k nearest neighbors
    distances, neighbours = k_neighbours(X_train, X_test, k, lamb, return_distance=True)

    # Compute inverse of distances
    epsilon = 1e-8
    inverse_dist = 1 / (distances + epsilon)  # Avoid division by zero

    # Weights assigned to every neighbour depending on which class they are
    class_based_weights = np.array([[class_weights[y_train[index]] for index in row] for row in neighbours])

    total_weights = inverse_dist * class_based_weights  # Combine distance and class-based weights

    # Count occurrences of labels with np.bincount and choose the label with the highest weighted count
    y_pred = np.array([
        np.argmax(np.bincount(y_train[neighbours[i]], weights=total_weights[i])) 
        for i in range(len(neighbours))
    ]) 

    return y_pred

In [66]:
weighted_prediction = predict_kNN_weight(X_train.to_numpy(), y_train.to_numpy(), X_test.to_numpy(), optimal_k, optimal_lamb, class_weights)

In [67]:
# Confusion matrix for prediction with arbitrary weights

confusion_matrix(y_test, weighted_prediction)

Predicted,0,1,2
Truth,,,
0,9,9,0
1,4,31,0
2,0,3,4


## Tuning $k$, $\lambda$ and the Weights via T-fold Cross Validation

In [69]:
def cross_validation_score_kNN_weighted(X_train, y_train, folds_indexes, k, lamb, class_weights):

    # Store the scores for each fold
    scores = [] 

    # Iterate over the folds
    for i in range(len(folds_indexes)): 
        # The indices for that fold (what we will validate on)
        val_indexes = folds_indexes[i] 

        # Get the indices of all the other folds (what we will train on)
        train_indexes = list(set(range(y_train.shape[0])) - set(val_indexes)) 

        # Get the training data
        X_train_i = X_train[train_indexes, :] 
        y_train_i = y_train[train_indexes]

        # Get the validation data
        X_val_i = X_train[val_indexes, :] 
        y_val_i = y_train[val_indexes] 

        # Make a prediction on the validation fold
        prediction = predict_kNN_weight(X_train_i, y_train_i, X_val_i, k, lamb, class_weights)

        # Store the macro-averaged recall score for this prediction
        score_i = macro_recall(y_val_i, prediction)
        scores.append(score_i)

    # Return the average score
    return sum(scores) / len(scores) 

In [70]:
def choose_best_k_and_lamb_and_weights(X_train, y_train, folds_indexes, k_range, lamb_range, weight0_range, weight1_range, weight2_range):
    best_score = 0

    # Loop over the ranges of every hyperparameter
    for i, k in enumerate(k_range):
        for j, lamb in enumerate(lamb_range):
            for l, w0 in enumerate(weight0_range):
                for n, w1 in enumerate(weight1_range):
                    for m, w2 in enumerate(weight2_range):
                        class_weights = {0: w0, 1: w1, 2: w2}
                        score = cross_validation_score_kNN_weighted(X_train, y_train, folds_indexes, k, lamb, class_weights)
                        if score > best_score:
                            best_score = score
                            best_params = [k,lamb,w0,w1,w2]
                        #print(f'The cross validation score for the values {k,lamb,w0,w1,w2}: {score:.3f}')
                        
    
    return best_score, best_params

In [71]:
# Selecting the best values for all hyperparameters

k_range = np.arange(4, 9) #4,...,8
lamb_range = np.arange(0.1,1.1,0.1) #0.1,0.2,...,1
w0_range = np.arange(0.1,1.1,0.1)
w1_range = np.arange(0.1,1.1,0.1)
w2_range = np.arange(0.1,1.1,0.1)

best_values = choose_best_k_and_lamb_and_weights(X_train.to_numpy(), y_train.to_numpy(), folds_indexes, k_range, lamb_range, w0_range, w1_range, w2_range)
print(f'Best set of values is {best_values[1]} with score {best_values[0]}.')

Best set of values is [6, 0.6, 0.2, 0.1, 0.30000000000000004] with score 0.8190752132857396.


In [72]:
# Collect optimal values for the hyperparameters

optimal_k, optimal_lamb, optimal_w0, optimal_w1, optimal_w2 = best_values[1][0], best_values[1][1], best_values[1][2], best_values[1][3], best_values[1][4]
optimal_class_weights = {0: optimal_w0, 1: optimal_w1, 2: optimal_w2}

## Discussion

The fact that the optimal weights are $0.1,0.2,0.3$ for Classes $1,0,2$ is telling: Those with a smaller amount of samples got assigned greater weighting to counteract this lack of samples. We can compare the confusion matrix of this weighted method with the original unweighted one:

In [75]:
# Confusion matrix for the prediction with optimal weights

optimal_weighted_prediction = predict_kNN_weight(X_train.to_numpy(), y_train.to_numpy(), X_test.to_numpy(), optimal_k, optimal_lamb, optimal_class_weights)
confusion_matrix(y_test, optimal_weighted_prediction)

Predicted,0,1,2
Truth,,,
0,11,6,1
1,7,28,0
2,0,2,5


In [76]:
# Confusion matrix we had for the original prediction without weighting

confusion_matrix(y_test, optimal_prediction)

Predicted,0,1,2
Truth,,,
0,12,6,0
1,6,29,0
2,0,3,4


There is not much difference. However, one can tell that Class $2$ was given high importance (it is better predicted), while Class $1$ was given less weighting (the differentiation between Classes $0,1$ got blurrier). Overall, this was a marked improvement towards the second goal of predicting the minority class accurately, but at the cost of the first goal. It is clear that both goals are competing, and achieving both at the same time is very difficult without a lot more data. In order to simplify the task for the model, we can first complete the first goal and then the second **separately**. 

**Improvement:** We will try to achieve both goals with a **two-step kNN**. We first use a weighted kNN to distinguish between Class $0$ and Classes $1,2$ (first goal). Then we will use another weighted kNN on the malignant set to distinguish between Classes $1$ and $2$ (second goal). This should be a model that predicts all classes better.

# Two-step kNN

## Implementing Two-step kNN

In [81]:
def predict_twostep_kNN(X_train, y_train, X_test, k1, k2, lamb1, lamb2, weights1, weights2):

    
    # FIRST STEP: 
    # We relabel Classes $1,2$ as just Class $1$ and then we use a simple kNN
    # to distinguish this new class from Class $0$. This separates benign and malignant cases.

    # Store the original classes
    indices_0 = np.where(y_train==0)[0]
    indices_1 = np.where(y_train==1)[0]
    indices_2 = np.where(y_train==2)[0]

    # Relabel Class 2 as Class 1
    modified_y_train = np.copy(y_train)
    modified_y_train[indices_2] = 1

    # Make the prediction that separates benign from malignant 
    first_prediction_round = predict_kNN_weight(X_train, modified_y_train, X_test, k=k1, lamb=lamb1, class_weights=weights1)



    # SECOND STEP:
    # Make a prediction on the samples which were predicted to be malignant in the first step to distinguish Classes 1 and 2.

    # Store which samples were classed as malignant or benign
    benign_ind = np.where(first_prediction_round == 0)[0]
    malignant_ind = np.where(first_prediction_round == 1)[0]

    # Remove the samples which were predicted to be benign from the test set 
    new_X_test = np.delete(X_test, benign_ind, axis=0)

    # Removing the Class 0 samples from the training data
    new_X_train = np.delete(X_train, indices_0, axis=0)
    new_y_train = np.delete(y_train, indices_0)

    # Make the prediction that separates the two types of malignant
    second_prediction_round = predict_kNN_weight(new_X_train, new_y_train, new_X_test, k=k2, lamb=lamb2, class_weights=weights2)



    # Chain both predictions together

    # First prediction
    final_prediction = np.copy(first_prediction_round)

    # Second prediction
    final_prediction[malignant_ind] = second_prediction_round
    

    return final_prediction

## Tuning $k_1,k_2,\lambda_1, \lambda_2$ and the Weights

In [83]:
# Compute the cross validation score for given values of k_1,k_2,lambda_1 and lambda_2

def cross_validation_score_twostep_kNN(X_train, y_train, folds_indexes, k1, k2, lamb1, lamb2, weights1, weights2):

    # Store the scores for each fold
    scores = [] 

    # Iterate over the folds
    for i in range(len(folds_indexes)): 
        # The indices for that fold (what we will validate on)
        val_indexes = folds_indexes[i] 

        # Get the indices of all the other folds (what we will train on)
        train_indexes = list(set(range(y_train.shape[0])) - set(val_indexes)) 

        # Get the training data
        X_train_i = X_train[train_indexes, :] 
        y_train_i = y_train[train_indexes]

        # Get the validation data
        X_val_i = X_train[val_indexes, :] 
        y_val_i = y_train[val_indexes] 

        # Make a prediction on the validation fold 
        prediction = predict_twostep_kNN(X_train_i, y_train_i, X_val_i, k1, k2, lamb1, lamb2, weights1, weights2)

        # Store the macro-averaged recall score for this prediction
        score_i = macro_recall(y_val_i, prediction)
        scores.append(score_i)

    # Return the average score
    return sum(scores) / len(scores)

In [84]:
def choose_best_params(X_train, y_train, folds_indexes, k1_range, k2_range, lamb1_range, lamb2_range, w10_range, w11_range, w21_range, w22_range):
    best_score = 0
    
    # Loop over the ranges of every hyperparameter
    for i, k1 in enumerate(k1_range):
        for j, k2 in enumerate(k2_range):
            for l, lamb1 in enumerate(lamb1_range):
                for n, lamb2 in enumerate(lamb2_range):
                    for m, w10 in enumerate(w10_range):
                        for p, w11 in enumerate(w11_range):
                            for q, w21 in enumerate(w21_range):
                                for s, w22 in enumerate(w22_range):
                                    weights1 = {0:w10,1:w11}
                                    weights2 = {1:w21, 2:w22}
                                    score = cross_validation_score_twostep_kNN(X_train, y_train, folds_indexes, k1, k2, lamb1, lamb2, weights1, weights2)
                                    if score > best_score:
                                        best_score = score
                                        best_params = [k1,k2,lamb1,lamb2,w10,w11,w21,w22]
                                    #print(f'The cross validation score for the values {k1,k2,lamb1,lamb2,w10,w11,w21,w22}: {score:.3f}')

    
    return best_score, best_params

In [85]:
# Selecting the best values for all hyperparameters

k1_range = np.arange(3, 9) #3,...,8
k2_range = np.arange(1, 6) #1,...,5
lamb1_range = np.arange(0.2,1,0.1) #0.1,0.2,...,0.9
lamb2_range = np.arange(0.2,1,0.1) 
w10_range = np.arange(0.1,0.6,0.1)
w11_range = np.arange(0.1,0.5,0.1)
w21_range = np.arange(0.1,0.5,0.1)
w22_range = np.arange(0.1,0.6,0.1)

best_values = choose_best_params(X_train.to_numpy(), y_train.to_numpy(), folds_indexes, k1_range, k2_range, lamb1_range, lamb2_range, w10_range, w11_range, w21_range, w22_range)
print(f'Best set of values is {best_values[1]} with score {best_values[0]}.')

KeyboardInterrupt: 

In [ ]:
# Collect optimal values for the hyperparameters

optimal_k1, optimal_k2, optimal_lamb1, optimal_lamb2 = best_values[1][0], best_values[1][1], best_values[1][2], best_values[1][3]
optimal_w10, optimal_w11, optimal_w21, optimal_w22 = best_values[1][4], best_values[1][5], best_values[1][6], best_values[1][7]

optimal_weights1 = {0:optimal_w10,1:optimal_w11}
optimal_weights2 = {0:optimal_w21,1:optimal_w22}

## Discussion

In [ ]:
optimal_twostep_prediction = predict_twostep_kNN(X_train.to_numpy(), y_train.to_numpy(), X_test.to_numpy(), optimal_k1, optimal_k2, optimal_lamb1, optimal_lamb2, optimal_weights1, optimal_weights2)
confusion_matrix(y_test, optimal_twostep_prediction)